## Imports


In [ ]:
# imports
import os
import shutil
import json
import random
import kaggle
import torch
from torchvision import transforms
from torchvision import datasets
from torchvision import models
from torchsummary import summary
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image


## Prepare data

In [ ]:
# paths
tmp_path = 'tmp'
train_dataset_path = f'{tmp_path}/train_dataset'
test_dataset_path = f'{tmp_path}/test_dataset'


In [ ]:
# download dataset
if not os.path.exists(train_dataset_path) or not os.path.exists(test_dataset_path):
    kaggle.api.authenticate()
    kaggle.api.dataset_download_files('grassknoted/asl-alphabet', path=tmp_path, unzip=True)

    shutil.move(f"{tmp_path}/asl_alphabet_train/asl_alphabet_train", train_dataset_path)
    shutil.move(f"{tmp_path}/asl_alphabet_test/asl_alphabet_test", test_dataset_path)

    shutil.rmtree(f"{tmp_path}/asl_alphabet_train")
    shutil.rmtree(f"{tmp_path}/asl_alphabet_test")


## Configure PyTorch


In [ ]:
# PyTorch device
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


## Prepare data


In [ ]:
# Define transform
transform = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
transform


In [ ]:
# Load dataset
dataset = datasets.ImageFolder(root=train_dataset_path, transform=transform)
dataset


In [ ]:
# Get classes
classes = dataset.classes
classes


In [ ]:
# Count classes
classes_count = len(classes)
classes_count


In [ ]:
# Set train/test split ratio
train_ratio = 0.8
test_ratio = 0.2
{"train_ratio": train_ratio, "test_ratio": test_ratio}


In [ ]:
# Set train/test sizes
train_size = int(train_ratio * len(dataset))
test_size = int(test_ratio * len(dataset))
{"train_size": train_size, "test_size": test_size}


In [ ]:
# Split dataset into train and test
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])


In [ ]:
# Count train/test dataset sizes
train_dataset_count = len(train_dataset)
test_dataset_count = len(test_dataset)
{"train_dataset_count": train_dataset_count, "test_dataset_count": test_dataset_count}


In [ ]:
# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True)


In [ ]:
# Display random images function
def show_random_samples(dataset, classes, num_samples=5):
    fig, axs = plt.subplots(1, num_samples, figsize=(15, 15))

    for i in range(num_samples):
        index = np.random.randint(0, len(dataset) - 1)
        image, label = dataset[index]

        axs[i].imshow(image.permute(1, 2, 0))
        axs[i].set_title(classes[label])
        axs[i].axis('off')

    plt.show()


In [ ]:
# Display random images from train dataset
show_random_samples(train_dataset, classes)


In [ ]:
# Display random images from test dataset
show_random_samples(test_dataset, classes)


## Prepare models


### Make models folder

In [ ]:
os.makedirs('tmp/models', exist_ok=True)


## Make model losses folder


In [ ]:
os.makedirs('tmp/model_losses', exist_ok=True)


### AlexNet


In [ ]:
AlexNet_model = models.alexnet()

# Freeze model parameters
for param in AlexNet_model.parameters():
    param.requires_grad = False

# Modify the last fully connected layer to fit your number of classes
AlexNet_model.classifier[6] = torch.nn.Linear(AlexNet_model.classifier[6].in_features, classes_count)

# Move the model to the device (GPU if available)
AlexNet_model.to(device)

# Define loss function and optimizer
AlexNet_model_criterion = torch.nn.CrossEntropyLoss()
AlexNet_model_optimizer = torch.optim.Adam(AlexNet_model.parameters(), lr=0.001)


In [ ]:
summary(AlexNet_model, input_size=(3, 200, 200))


### GoogleNet


In [ ]:
GoogleNet_model = models.googlenet(models.GoogLeNet_Weights.DEFAULT)

# Freeze model parameters
for param in GoogleNet_model.parameters():
    param.requires_grad = False

# Modify the last fully connected layer to fit your number of classes
GoogleNet_model.fc = torch.nn.Linear(GoogleNet_model.fc.in_features, classes_count)

# Move the model to the device (GPU if available)
GoogleNet_model.to(device)

# Define loss function and optimizer
GoogleNet_model_criterion = torch.nn.CrossEntropyLoss()
GoogleNet_model_optimizer = torch.optim.Adam(GoogleNet_model.parameters(), lr=0.001)


In [ ]:
summary(GoogleNet_model, input_size=(3, 200, 200))


### MobileNet V2


In [ ]:
# MobileNetV2_model = models.MobileNetV2()

# # Freeze model parameters
# for param in MobileNetV2_model.parameters():
#     param.requires_grad = False

# # Modify the last fully connected layer to fit your number of classes
# MobileNetV2_model.classifier[6] = torch.nn.Linear(MobileNetV2_model.classifier[6].in_features, classes_count)

# # Move the model to the device (GPU if available)
# MobileNetV2_model.to(device)

# # Define loss function and optimizer
# MobileNetV2_model_criterion = torch.nn.CrossEntropyLoss()
# MobileNetV2_model_optimizer = torch.optim.Adam(MobileNetV2_model.parameters(), lr=0.001)


In [ ]:
# summary(MobileNetV2_model, input_size=(3, 200, 200))


### MobileNet V3


In [ ]:
# MobileNetV3_model = models.MobileNetV3()

# # Freeze model parameters
# for param in MobileNetV3_model.parameters():
#     param.requires_grad = False

# # Modify the last fully connected layer to fit your number of classes
# MobileNetV3_model.classifier[6] = torch.nn.Linear(MobileNetV3_model.classifier[6].in_features, classes_count)

# # Move the model to the device (GPU if available)
# MobileNetV3_model.to(device)

# # Define loss function and optimizer
# MobileNetV3_model_criterion = torch.nn.CrossEntropyLoss()
# MobileNetV3_model_optimizer = torch.optim.Adam(MobileNetV3_model.parameters(), lr=0.001)


In [ ]:
# summary(MobileNetV3_model, input_size=(3, 200, 200))


### ResNet 18


In [ ]:
ResNet18_model = models.resnet18()

# Freeze model parameters
for param in ResNet18_model.parameters():
    param.requires_grad = False

# Modify the last fully connected layer to fit your number of classes
ResNet18_model.fc = torch.nn.Linear(ResNet18_model.fc.in_features, classes_count)

# Move the model to the device (GPU if available)
ResNet18_model.to(device)

# Define loss function and optimizer
ResNet18_model_criterion = torch.nn.CrossEntropyLoss()
ResNet18_model_optimizer = torch.optim.Adam(ResNet18_model.parameters(), lr=0.001)


In [ ]:
summary(ResNet18_model, input_size=(3, 200, 200))


### ResNet 50


In [ ]:
ResNet50_model = models.resnet50()

# Freeze model parameters
for param in ResNet50_model.parameters():
    param.requires_grad = False

# Modify the last fully connected layer to fit your number of classes
ResNet50_model.fc = torch.nn.Linear(ResNet50_model.fc.in_features, classes_count)

# Move the model to the device (GPU if available)
ResNet50_model.to(device)

# Define loss function and optimizer
ResNet50_model_criterion = torch.nn.CrossEntropyLoss()
ResNet50_model_optimizer = torch.optim.Adam(ResNet50_model.parameters(), lr=0.001)


In [ ]:
summary(ResNet50_model, input_size=(3, 200, 200))


### Vgg16


In [ ]:
Vgg16_model = models.vgg16()

# Freeze model parameters
for param in Vgg16_model.parameters():
    param.requires_grad = False

# Modify the last fully connected layer to fit your number of classes
Vgg16_model.classifier[6] = torch.nn.Linear(Vgg16_model.classifier[6].in_features, classes_count)

# Move the model to the device (GPU if available)
Vgg16_model.to(device)

# Define loss function and optimizer
Vgg16_model_criterion = torch.nn.CrossEntropyLoss()
Vgg16_model_optimizer = torch.optim.Adam(Vgg16_model.parameters(), lr=0.001)


In [ ]:
summary(Vgg16_model, input_size=(3, 200, 200))


### Vgg19


In [ ]:
Vgg19_model = models.vgg16()

# Freeze model parameters
for param in Vgg19_model.parameters():
    param.requires_grad = False

# Modify the last fully connected layer to fit your number of classes
Vgg19_model.classifier[6] = torch.nn.Linear(Vgg19_model.classifier[6].in_features, classes_count)

# Move the model to the device (GPU if available)
Vgg19_model.to(device)

# Define loss function and optimizer
Vgg19_model_criterion = torch.nn.CrossEntropyLoss()
Vgg19_model_optimizer = torch.optim.Adam(Vgg19_model.parameters(), lr=0.001)


In [ ]:
summary(Vgg19_model, input_size=(3, 200, 200))


### Custom


## Train models


In [ ]:
# train model function
def train_model(model, train_loader, criterion, optimizer, device, num_epochs=10):
    epochs_losses = []
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * inputs.size(0)
        epoch_loss /= len(train_loader.dataset)
        epochs_losses.append(epoch_loss)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")
    return epochs_losses


### AlexNet


In [ ]:
AlexNet_model_losses = None
model_path = 'tmp/models/ASL_AlexNet_model_V1.0.0.pth'
model_losses_path = 'tmp/model_losses/ASL_AlexNet_model_losses_V1.0.0.json'

if os.path.isfile(model_path):
    AlexNet_model = torch.load(model_path)

    with open(model_losses_path) as filehandle:
        AlexNet_model_losses = json.load(filehandle)
else:
    AlexNet_model_losses = train_model(AlexNet_model, train_loader, AlexNet_model_criterion, AlexNet_model_optimizer, device, num_epochs=10)
    torch.save(AlexNet_model, model_path)

    with open(model_losses_path, 'w') as filehandle:
        json.dump(AlexNet_model_losses, filehandle)


In [ ]:
AlexNet_model_losses


### GoogleNet


In [ ]:
GoogleNet_model_losses = None
model_path = 'tmp/models/ASL_GoogleNet_model_V1.0.0.pth'
model_losses_path = 'tmp/model_losses/ASL_GoogleNet_model_losses_V1.0.0.json'

if os.path.isfile(model_path):
    GoogleNet_model = torch.load(model_path)

    with open(model_losses_path) as filehandle:
        GoogleNet_model_losses = json.load(filehandle)
else:
    GoogleNet_model_losses = train_model(GoogleNet_model, train_loader, GoogleNet_model_criterion, GoogleNet_model_optimizer, device, num_epochs=10)
    torch.save(GoogleNet_model, model_path)

    with open(model_losses_path, 'w') as filehandle:
        json.dump(GoogleNet_model_losses, filehandle)


In [ ]:
GoogleNet_model_losses


### MobileNet V2


In [ ]:
# MobileNetV2_model_losses = None
# model_path = 'tmp/models/ASL_MobileNetV2_model_V1.0.0.pth'
# model_losses_path = 'tmp/model_losses/ASL_MobileNetV2_model_losses_V1.0.0.json'
# 
# if os.path.isfile(model_path):
#     MobileNetV2_model = torch.load(model_path)
# 
#     with open(model_losses_path) as filehandle:
#         MobileNetV2_model_losses = json.load(filehandle)
# else:
#     MobileNetV2_model_losses = train_model(MobileNetV2_model, train_loader, MobileNetV2_model_criterion, MobileNetV2_model_optimizer, device, num_epochs=10)
#     torch.save(MobileNetV2_model, model_path)
# 
#     with open(model_losses_path, 'w') as filehandle:
#         json.dump(MobileNetV2_model_losses, filehandle)


In [ ]:
# MobileNetV2_model_losses


### MobileNet V3


In [ ]:
# MobileNetV3_model_losses = None
# model_path = 'tmp/models/ASL_MobileNetV3_model_V1.0.0.pth'
# model_losses_path = 'tmp/model_losses/ASL_MobileNetV3_model_losses_V1.0.0.json'
# 
# if os.path.isfile(model_path):
#     MobileNetV3_model = torch.load(model_path)
# 
#     with open(model_losses_path) as filehandle:
#         MobileNetV3_model_losses = json.load(filehandle)
# else:
#     MobileNetV3_model_losses = train_model(MobileNetV3_model, train_loader, MobileNetV3_model_criterion, MobileNetV3_model_optimizer, device, num_epochs=10)
#     torch.save(MobileNetV3_model, model_path)
# 
#     with open(model_losses_path, 'w') as filehandle:
#         json.dump(MobileNetV3_model_losses, filehandle)


In [ ]:
# MobileNetV3_model_losses


### ResNet 18


In [ ]:
ResNet18_model_losses = None
model_path = 'tmp/models/ASL_ResNet18_model_V1.0.0.pth'
model_losses_path = 'tmp/model_losses/ASL_ResNet18_model_losses_V1.0.0.json'

if os.path.isfile(model_path):
    ResNet18_model = torch.load(model_path)

    with open(model_losses_path) as filehandle:
        ResNet18_model_losses = json.load(filehandle)
else:
    ResNet18_model_losses = train_model(ResNet18_model, train_loader, ResNet18_model_criterion, ResNet18_model_optimizer, device, num_epochs=10)
    torch.save(ResNet18_model, model_path)

    with open(model_losses_path, 'w') as filehandle:
        json.dump(ResNet18_model_losses, filehandle)


In [ ]:
ResNet18_model_losses


### ResNet 50


In [ ]:
ResNet50_model_losses = None
model_path = 'tmp/models/ASL_ResNet50_model_V1.0.0.pth'
model_losses_path = 'tmp/model_losses/ASL_ResNet50_model_losses_V1.0.0.json'

if os.path.isfile(model_path):
    ResNet50_model = torch.load(model_path)

    with open(model_losses_path) as filehandle:
        ResNet50_model_losses = json.load(filehandle)
else:
    ResNet50_model_losses = train_model(ResNet50_model, train_loader, ResNet50_model_criterion, ResNet50_model_optimizer, device, num_epochs=10)
    torch.save(ResNet50_model, model_path)

    with open(model_losses_path, 'w') as filehandle:
        json.dump(ResNet50_model_losses, filehandle)


In [ ]:
ResNet50_model_losses


### Vgg16


In [ ]:
Vgg16_model_losses = None
model_path = 'tmp/models/ASL_Vgg16_model_V1.0.0.pth'
model_losses_path = 'tmp/model_losses/ASL_Vgg16_model_losses_V1.0.0.json'

if os.path.isfile(model_path):
    Vgg16_model = torch.load(model_path)

    with open(model_losses_path) as filehandle:
        Vgg16_model_losses = json.load(filehandle)
else:
    Vgg16_model_losses = train_model(Vgg16_model, train_loader, Vgg16_model_criterion, Vgg16_model_optimizer, device, num_epochs=10)
    torch.save(Vgg16_model, model_path)

    with open(model_losses_path, 'w') as filehandle:
        json.dump(Vgg16_model_losses, filehandle)


In [ ]:
Vgg16_model_losses


### Vgg19


In [ ]:
Vgg19_model_losses = None
model_path = 'tmp/models/ASL_Vgg19_model_V1.0.0.pth'
model_losses_path = 'tmp/model_losses/ASL_Vgg19_model_losses_V1.0.0.json'

if os.path.isfile(model_path):
    Vgg19_model = torch.load(model_path)

    with open(model_losses_path) as filehandle:
        Vgg19_model_losses = json.load(filehandle)
else:
    Vgg19_model_losses = train_model(Vgg19_model, train_loader, Vgg19_model_criterion, Vgg19_model_optimizer, device, num_epochs=10)
    torch.save(Vgg19_model, model_path)

    with open(model_losses_path, 'w') as filehandle:
        json.dump(Vgg19_model_losses, filehandle)


In [ ]:
Vgg19_model_losses


### Custom


In [ ]:
# Custom_model_losses = None
# model_path = 'tmp/models/ASL_Custom_model_V1.0.0.pth'
# model_losses_path = 'tmp/model_losses/ASL_Custom_model_losses_V1.0.0.json'
# 
# if os.path.isfile(model_path):
#     Custom_model = torch.load(model_path)
# 
#     with open(model_losses_path) as filehandle:
#         Custom_model_losses = json.load(filehandle)
# else:
#     Custom_model_losses = train_model(Custom_model, train_loader, Custom_model_criterion, Custom_model_optimizer, device, num_epochs=10)
#     torch.save(Custom_model, model_path)
# 
#     with open(model_losses_path, 'w') as filehandle:
#         json.dump(Custom_model_losses, filehandle)


In [ ]:
# Custom_model_losses


### Losses plot


In [ ]:
all_losses = [
    AlexNet_model_losses,
    GoogleNet_model_losses,
    # MobileNetV2_model_losses,
    # MobileNetV3_model_losses,
    ResNet18_model_losses,
    ResNet50_model_losses,
    Vgg16_model_losses,
    Vgg19_model_losses,
    # Custom_model_losses,
]

plt.figure(figsize=(10, 5))
for losses in all_losses:
    plt.plot(losses)
plt.title('Model Losses')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend([
    "AlexNet",
    "GoogleNet",
    # "MobileNetV2",
    # "MobileNetV3",
    "ResNet18",
    "ResNet50",
    "Vgg16",
    "Vgg19",
    # "Custom",
])
plt.show()


## Evaluate


In [ ]:
# test model function
def test_model(model, test_loader, criterion, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total * 100
    print(f"Test Accuracy: {accuracy:.2f}%")
    return accuracy


### AlexNet


In [ ]:
# test the model
AlexNet_model_accuracy = test_model(AlexNet_model, test_loader, AlexNet_model_criterion, device)


### GoogleNet


In [ ]:
# test the model
GoogleNet_model_accuracy = test_model(GoogleNet_model, test_loader, GoogleNet_model_criterion, device)


### MobileNet V2


In [ ]:
# test the model
# MobileNetV2_model_accuracy = test_model(MobileNetV2_model, test_loader, MobileNetV2_model_criterion, device)


### MobileNet V3


In [ ]:
# test the model
# MobileNetV3_model_accuracy = test_model(MobileNetV3_model, test_loader, MobileNetV3_model_criterion, device)


### ResNet 18


In [ ]:
# test the model
ResNet18_model_accuracy = test_model(ResNet18_model, test_loader, ResNet18_model_criterion, device)


### ResNet 50


In [ ]:
# test the model
ResNet50_model_accuracy = test_model(ResNet50_model, test_loader, ResNet50_model_criterion, device)


### Vgg16


In [ ]:
# test the model
Vgg16_model_accuracy = test_model(Vgg16_model, test_loader, Vgg16_model_criterion, device)


### Vgg19


In [ ]:
# test the model
Vgg19_model_accuracy = test_model(Vgg19_model, test_loader, Vgg19_model_criterion, device)


### Custom


In [ ]:
# test the model
# Custom_model_accuracy = test_model(Custom_model, test_loader, Custom_model_criterion, device)


### Summary


In [ ]:
# Create a dictionary to store results
model_results = {
    "Model": [
        "AlexNet",
        "GoogleNet",
        # "MobileNetV2",
        # "MobileNetV3",
        "ResNet18",
        "ResNet50",
        "Vgg16",
        "Vgg19",
        # "Custom",
    ],
    "Loss": [
        AlexNet_model_losses[-1],
        GoogleNet_model_losses[-1],
        # MobileNetV2_model_losses[-1],
        # MobileNetV3_model_losses[-1],
        ResNet18_model_losses[-1],
        ResNet50_model_losses[-1],
        Vgg16_model_losses[-1],
        Vgg19_model_losses[-1],
        # Custom_model_losses[-1],
    ],
    "Accuracy": [
        AlexNet_model_accuracy,
        GoogleNet_model_accuracy,
        # MobileNetV2_model_accuracy,
        # MobileNetV3_model_accuracy,
        ResNet18_model_accuracy,
        ResNet50_model_accuracy,
        Vgg16_model_accuracy,
        Vgg19_model_accuracy,
        # Custom_model_accuracy,
    ]
}

# Create a Pandas DataFrame from the dictionary
df = pd.DataFrame(model_results)

# Print the DataFrame as a table
print(df.to_string())


## Test


In [ ]:
# predict and display image function
def predict_and_display_image(model, image_path, transform, device):
    # Load the image
    image = Image.open(image_path).convert('RGB')

    # Preprocess the image
    image_tensor = transform(image).unsqueeze(0).to(device)

    # Set the model to evaluation mode
    model.eval()

    with torch.no_grad():
        # Make a prediction
        outputs = model(image_tensor)
        _, predicted = torch.max(outputs, 1)
        predicted_class = classes[predicted.item()]

    # Display the image and prediction
    plt.imshow(image)
    plt.title(f"Predicted: {predicted_class} - {model.__class__.__name__}")
    plt.axis('off')
    plt.show()


In [ ]:
images_paths = [
    'tmp/test_g.jpeg',
    'tmp/test_dataset/G_test.jpg',
]


### AlexNet


In [ ]:
for image_path in images_paths:
    predict_and_display_image(AlexNet_model, image_path, transform, device)


### GoogleNet


In [ ]:
for image_path in images_paths:
    predict_and_display_image(GoogleNet_model, image_path, transform, device)


### MobileNet V2


In [ ]:
# for image_path in images_paths:
#     predict_and_display_image(MobileNetV2_model, image_path, transform, device)


### MobileNet V3


In [ ]:
# for image_path in images_paths:
#     predict_and_display_image(MobileNetV3_model, image_path, transform, device)


### ResNet 18


In [ ]:
for image_path in images_paths:
    predict_and_display_image(ResNet18_model, image_path, transform, device)


### ResNet 50


In [ ]:
for image_path in images_paths:
    predict_and_display_image(ResNet50_model, image_path, transform, device)


### Vgg16


In [ ]:
for image_path in images_paths:
    predict_and_display_image(Vgg16_model, image_path, transform, device)


### Vgg19


In [ ]:
for image_path in images_paths:
    predict_and_display_image(Vgg19_model, image_path, transform, device)


### Custom


In [ ]:
# for image_path in images_paths:
#     predict_and_display_image(Custom_model, image_path, transform, device)
